In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.cif import CifData
from aiida.orm.calculation import Calculation

import ase.io
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk

from apps.lsmo.structure_browser import StructureBrowser


import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
node = False
structures = [("select structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms, node
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    if node is False:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

## Step 1: Select Structure

In [ ]:
def on_struct_change(c):
    global atoms, node
    node = struct_browser.results.value
    refresh_structure_view()
    if not node:
        return
    cell_params.value = "Unit cell <br />a = {:.2f} {:.2f} {:.2f} <br />b = {:.2f} {:.2f} {:.2f} <br />c = {:.2f} {:.2f} {:.2f}".format(
        atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
        atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
        atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()
cell_params = ipw.HTML("Cell: ")
clear_output()
display(ipw.VBox([struct_browser, viewer, cell_params]))

## Step 2: Use an AiiDA WorkChain to automatically compute the equation of states

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm import CalculationFactory, DataFactory
from aiida.orm.utils import load_node
from aiida.orm.code import Code
from aiida.orm.data.base import Float, Str
from aiida.orm.data.array import ArrayData
from aiida.orm.data.upf import UpfData
from aiida.orm.data.structure import StructureData

from aiida.common.exceptions import NotExistent
from aiida.work.run import run, submit
from aiida.work.workchain import WorkChain, ToContext, while_, Outputs

ParameterData = DataFactory('parameter')
RaspaCalculation = CalculationFactory('raspa')


%matplotlib notebook
import matplotlib.pyplot as plt


In [ ]:
# Initialize plot variables
fig, ax = plt.subplots(1,1)
garbage = ax.set_xlabel(u"Pressure [kPa]")
garbage = ax.set_ylabel(u"Loading average [molecules/unit cell]")

In [ ]:
class Isotherm(WorkChain):
    """
    Workchain that for a given matherial will compute
    """

    @classmethod
    def define(cls, spec):
        """
        This is the most important method of a Workchain, that defines the
        inputs that it takes, the logic of the execution and the outputs
        that are generated in the process 
        """
        super(Isotherm, cls).define(spec)
        
        # First we define the inputs, specifying the type we expect
        spec.input("pressures", valid_type=ArrayData, required=True)
        spec.input("structure", valid_type=CifData, required=True)
        spec.input("codename", valid_type=Str, required=True)
        
        # The outline describes the business logic that defines
        # which steps are executed in what order and based on
        # what conditions. Each `cls.method` is implemented below
        spec.outline(
            cls.init,
            while_(cls.should_run_raspa)(
                cls.run_raspa,
                cls.parse_raspa,
            ),
            cls.return_result,
        )
        
        # Here we define the output the Workchain will generate and
        # return. Dynamic output allows a variety of AiiDA data nodes
        # to be returned
        spec.dynamic_output()

    def init(self):
        """
        Initialize variables and the scales we want to compute
        """
        self.ctx.p = 0
        self.ctx.prev_pk = None
        self.ctx.pressures = self.inputs.pressures.get_array("pressures")
        self.ctx.result = []
        self.ctx.options = {
            "resources": {
                "num_machines": 1,
                "tot_num_mpiprocs": 1,
                "num_mpiprocs_per_machine": 1,
            },
            "max_wallclock_seconds": 30 * 60,
#            "queue_name":"serial",
        }
        self.ctx.parameters ={
            "GeneralSettings":
            {
            "SimulationType"                   : "MonteCarlo",
            "NumberOfCycles"                   : 5000,
            "NumberOfInitializationCycles"     : 2000,
            "PrintEvery"                       : 1000,
            "Forcefield"                       : "GenericZeolites",
            "Framework"                        : 0,
            "UnitCells"                        : "3 3 3",
            "HeliumVoidFraction"               : 0.29,
            "ExternalTemperature"              : 300.0,
            },
            "Component":
            [{
            "MoleculeName"                     : "methane",
            "MoleculeDefinition"               : "TraPPE",
            "TranslationProbability"           : 0.5,
            "ReinsertionProbability"           : 0.5,
            "SwapProbability"                  : 1.0,
            "CreateNumberOfMolecules"          : 0,
            }],
        }

        


    def should_run_raspa(self):
        """
        This is the main condition of the while loop, as defined
        in the outline of the Workchain. We only run another
        pw.x calculation if the current iteration is smaller than
        the total number of scale factors we want to compute
        """
        return self.ctx.p < len(self.ctx.pressures)

    def run_raspa(self):
        """
        This is the main function that will perform a pw.x
        calculation for the current scaling factor
        """
        pressure = self.ctx.pressures[self.ctx.p]
        self.ctx.parameters['GeneralSettings']['ExternalPressure'] = pressure
        if self.ctx.prev_pk is not None:
            self.ctx.parameters['GeneralSettings']['RestartFile'] = True
            self.ctx.parameters['restart_pk'] = self.ctx.prev_pk

        # Create the input dictionary
        inputs = {
            'code'       : Code.get_from_string(self.inputs.codename.value),
            'structure'  : self.inputs.structure,
            'parameters' : ParameterData(dict=self.ctx.parameters),
            '_options'   : self.ctx.options,
        }

        # Create the calculation process and launch it
        self.report("Running raspa for the pressure {}".format(pressure))
        process = RaspaCalculation.process()
        future  = submit(process, **inputs)

        self.ctx.p += 1
        self.ctx.prev_pk = future.pid
        #print(future)

        return ToContext(raspa=Outputs(future))

    def parse_raspa(self):
        """
        Extract the volume and total energy of the last completed PwCalculation
        """
        volume = self.ctx.parameters['GeneralSettings']['ExternalPressure']
        loading_average = self.ctx.raspa["component_0"].dict.loading_absolute_average
        self.ctx.result.append((volume, loading_average))        
        self.plot_data()

    def return_result(self):
        """
        Attach the results of the PwCalculations and the initial structure to the outputs
        """
        result = {
            "initial_structure": self.inputs.structure,
            "result": ParameterData(dict={"eos": self.ctx.result}),
        }

        for link_name, node in result.iteritems():
            self.out(link_name, node)

        self.report("Workchain <{}> completed successfully".format(self.calc.pk))

        return
    def plot_data(self):
        ax.plot(*zip(*self.ctx.result), marker='o', linestyle='--', color='r')
        fig.canvas.draw()


In [ ]:
def get_code_options(plugin_classes):
    """
    Return AiiDA codes using a specific set of plugins
    
    :param plugin_classes: a dictionary of the type
      {'pw': 'quantumespresso.pw', 'ph': 'quantumespresso.ph'}
      where the key is a label and the value is the plugin to check for.
      It will return the set of codes that exist on the same machine.
    """
    from aiida.orm.querybuilder import QueryBuilder
    from aiida.orm import Code, Computer
    from aiida.backends.utils import get_automatic_user
    
    current_user = get_automatic_user()
    
    qb = QueryBuilder()
    qb.append(Computer,
          filters={'enabled': True},
          project=['*'], tag='computer')
    ordered_tags = []
    for tag, plugin_class in plugin_classes.iteritems():
        ordered_tags.append(tag)
        qb.append(Code,
          filters={'attributes.input_plugin': {'==': plugin_class},
                   'extras.hidden': {"~==": True}
            },
            project='label', tag='{}code'.format(tag), has_computer='computer')
    all_results = qb.all()
    # Filter in python only the ones that are actually user_configured
    # codeset[0] is the computer
    # codeset[1:] are the various code names, depending on the ones asked in input
    return [{tag: "{}@{}".format(codename, codeset[0].name) for codename, tag in zip(codeset[1:], ordered_tags)} 
            for codeset in all_results 
            if codeset[0].is_user_configured(current_user) and codeset[0].is_user_enabled(current_user)]

def get_code_dropdown(classes):
    """
    This function returns a group containing a dropdown list to select a
    valid available Quantum ESPRESSO pw.x code.

    To use it::

      code_group = get_code_pwonly_dropdown()


    You can later retrieve the value as follows::
   
      from IPython.display import display
      code_group = get_code_pwonly_dropdown()
      display(code_group)

    If this is None, then no code was found.
    Otherwise it will be a dictionary, where the only available key
    is 'pw' and the value is the code name, so you can get the code as::

       code_name = code_names['pw']
       code = Code.get_from_string(code_name)
    """
    import ipywidgets as ipw

    code_options_full = None
    in_codename = ipw.Dropdown(options=[], disabled=True)

    code_options_full = get_code_options(plugin_classes=classes)
    code_strings = ["{}".format(code_option['raspa']) 
        for code_option in code_options_full]  
        
    if code_options_full is None:
        in_codename.options=[["Error while retrieving the list of codes", None]]
        in_codename.disabled=True
        in_codename.value = None
    elif not code_options_full:
        in_codename.options = [["No AiiDA codes configured yet", None]]
        in_codename.disabled = True
        in_codename.value = None
    else:
        code_options = zip(code_strings, code_options_full)
        in_codename.options=code_options
        in_codename.disabled = False
        # Set default value (first entry)
        in_codename.value = code_options[0][1]    
                
    code_group = ipw.HBox(
        [
            ipw.Label(value="Select a quantum code to use: "), 
            in_codename,
        ])

    return code_group

code_group = get_code_dropdown(classes={'raspa': 'raspa'})

In [ ]:
def on_click_submit(b):
    btn_submit.disabled = True
    message.value = 'Please wait, the isotherm is being computed'
    if node is False:
        print ("Please select a structure")
        btn_submit.disabled = False
        return None
    codename = code_group.children[1].value['raspa']
    pressures = ArrayData()
    pressures.set_array("pressures", np.array([1e6, 2e6,3e6,4e6,5e6,6e6]))
    with submit_out:
        outputs = run(
            Isotherm,
            pressures=pressures,
            structure=node,
            codename=Str(codename),
        )

        message.value = "Final results of the \"isotherm\" workchain:"
        print ("{volume:12}  {loading_average:12}".format(volume="Volume (A^3)", loading_average="Loading average [molecules/unit cell]"))
        print ("{}".format("-"*26))
        for p in outputs["result"].get_dict()['eos']:
            print ("{volume:>12.5f}  {energy:>12.5f}".format(volume=p[0], energy=p[1]))
    btn_submit.disabled = False




btn_submit = ipw.Button(description='Submit Isotherm')
btn_submit.on_click(on_click_submit)
message = ipw.HTML('')
submit_out = ipw.Output()
display(ipw.HBox([code_group,btn_submit]), message, submit_out)